In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq gingado nbdev # install or upgrade gingado on colab

In [ ]:
#default_exp utils
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import show_doc

# Utils
> Functions to support the use of `gingado`

- badges: true

## Support for model documentation

In [ ]:
#hide
#export
import datetime
import os

> Important: Up until v0.0.1-11, function `get_username` existed. However, it was removed from `utils` since it depended on `pwd`, which is only available to Unix-like systems. Therefore, this [issue](https://github.com/dkgaraujo/gingado/issues/3) was preventing Windows users from importing `gingado.utils`. Since the function was not essential, it was removed until a suitable alternative that works in all major ystems can be found.

In [ ]:
#hide
#export
def get_datetime():
    "Returns the time now"
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S %Z") 

In [ ]:
show_doc(get_datetime)

<h4 id="get_datetime" class="doc_header"><code>get_datetime</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>get_datetime</code>()

Returns the time now

In [ ]:
d = get_datetime()
assert isinstance(d, str)
assert len(d) > 0

## Support for time series

Objects of the class `Lag` are similar to `scikit-learn`'s transformers.

In [ ]:
#hide
#export
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

class Lag(BaseEstimator, TransformerMixin):
    def __init__(self, lags=1, jump=0, keep_contemporaneous_X=False):
        self.lags = lags
        self.jump = jump
        self.keep_contemporaneous_X = keep_contemporaneous_X
    
    def fit(self, X, y=None):        
        self.index = None
        if hasattr(X, "index"):
            self.index = X.index
        else:
            if y is not None and hasattr(y, "index"):
                self.index = y.index
        X = self._validate_data(X)

        self.effective_lags_ = self.lags + self.jump
        return self

    def transform(self, X):
        X_forlag = X
        
        X = self._validate_data(X)
        check_is_fitted(self)
        X_lags = []
        X_colnames = list(self.feature_names_in_) if self.keep_contemporaneous_X else []
        for lag in range(self.effective_lags_):
            if lag < self.jump:
                continue
            lag_count = lag+1
            lag_X = np.roll(X_forlag, lag_count, axis=0)
            X_lags.append(lag_X)
            if hasattr(self, "feature_names_in_"):
                X_colnames = X_colnames + [col+"_lag_"+str(lag+1) for col in list(self.feature_names_in_)]
        X = np.concatenate(X_lags, axis=1)
        if self.keep_contemporaneous_X:
            X = np.concatenate([X_forlag, X], axis=1)
        X = X[self.effective_lags_:, :]
        if hasattr(self, "index") and self.index is not None:
            new_index = self.index[self.effective_lags_:]
            X = pd.DataFrame(X, index=new_index, columns=X_colnames)
        else:
            X = pd.DataFrame(X)
        return X

In [ ]:
show_doc(Lag)

<h2 id="Lag" class="doc_header"><code>class</code> <code>Lag</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Lag</code>(**`lags`**=*`1`*, **`jump`**=*`0`*, **`keep_contemporaneous_X`**=*`False`*) :: `BaseEstimator`

Base class for all estimators in scikit-learn.

Notes
-----
All estimators should specify all the parameters that can be set
at the class level in their ``__init__`` as explicit keyword
arguments (no ``*args`` or ``**kwargs``).

The code below demonstrates how `Lag` works in practice. Note in particular that, because `Lag` is a transformer, it can be used as part of a `scikit-learn`'s `Pipeline`. 

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

randomX = np.random.rand(15, 2)
randomY = np.random.rand(15)

lags = 3
jump = 2

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lagger', Lag(lags=lags, jump=jump, keep_contemporaneous_X=False))
]).fit_transform(randomX, randomY)

Below we confirm that the lagger removes the correct number of rows corresponding to the lagged observations:

In [ ]:
assert randomX.shape[0] - lags - jump == pipe.shape[0]

And because `Lag` is a transformer, its parameters (`lags` and `jump`) can be calibrated using hyperparameter tuning to achieve the best performance for a model.

## Support for data augmentation with SDMX

> Note: please note that working with SDMX may take some minutes depending on the amount of information you are downloading.

In [ ]:
#hide
#export
import pandasdmx as sdmx

def list_SDMX_sources():
    "Returns the list of codes representing the SDMX sources available for data download"
    return sdmx.list_sources()

In [ ]:
show_doc(list_SDMX_sources)

<h4 id="list_SDMX_sources" class="doc_header"><code>list_SDMX_sources</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>list_SDMX_sources</code>()

Returns the list of codes representing the SDMX sources available for data download

In [ ]:
sources = list_SDMX_sources()
print(sources)

assert len(sources) > 0
# all elements are of type 'str'
assert sum([isinstance(src, str) for src in sources]) == len(sources)

['ABS', 'ABS_XML', 'BBK', 'BIS', 'CD2030', 'ECB', 'ESTAT', 'ILO', 'IMF', 'INEGI', 'INSEE', 'ISTAT', 'LSD', 'NB', 'NBB', 'OECD', 'SGR', 'SPC', 'STAT_EE', 'UNICEF', 'UNSD', 'WB', 'WB_WDI']


In [ ]:
#hide
#export
import pandasdmx as sdmx

def list_all_dataflows(codes_only=False):
    "Returns a dictionary listing all available dataflows for all sources. When using as a parameter to an `AugmentSDMX` object or to the `load_SDMX_data` function, set `codes_only=True`"
    sources = sdmx.list_sources()
    dflows = {}
    for src in sources:
        try:
            dflows[src] = sdmx.to_pandas(sdmx.Request(src).dataflow().dataflow)
            dflows[src] = dflows[src].index if codes_only else dflows[src].index.reset_index()
        except:
            pass
    return dflows

In [ ]:
show_doc(list_all_dataflows)

<h4 id="list_all_dataflows" class="doc_header"><code>list_all_dataflows</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>list_all_dataflows</code>(**`codes_only`**=*`False`*)

Returns a dictionary listing all available dataflows for all sources. When using as a parameter to an [`AugmentSDMX`](/gingado/augmentation.html#AugmentSDMX) object or to the [`load_SDMX_data`](/gingado/utils.html#load_SDMX_data) function, set `codes_only=True`

In [ ]:
dflows = list_all_dataflows()

assert isinstance(dflows, dict)
all_sources = list_SDMX_sources()
assert len([s for s in dflows.keys() if s in all_sources]) == len(dflows.keys())

2022-05-31 22:58:08,594 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2022-05-31 22:59:46,534 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2022-05-31 22:59:53,245 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2022-05-31 22:59:57,240 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2022-05-31 22:59:58,367 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>


The function `load_SDMX_data` is a convenience function that downloads data from SDMX sources (and any specific dataflows passed as arguments) if they match the key and parameters set by the user.

In [ ]:
#hide
#export
import pandasdmx as sdmx

def load_SDMX_data(sources, keys, params, verbose=True):
    "Loads datasets from SDMX."
    data_sdmx = {}
    for source in sources.keys():
        src_conn = sdmx.Request(source)
        src_dflows = src_conn.dataflow()
        if sources[source] == 'all':
            dflows = {k: v for k, v in src_dflows.dataflow.items()}
        else:
            dflows = {k: v for k, v in src_dflows.dataflow.items() if k in sources[source]}
        for dflow in dflows.keys():
            if verbose: print(f"Querying data from {source}'s dataflow '{dflow}' - {dflows[dflow].dict()['name']}...")
            try:
                data = sdmx.to_pandas(src_conn.data(dflow, key=keys, params=params), datetime='TIME_PERIOD')
            except:
                if verbose: print("this dataflow does not have data in the desired frequency and time period.")
                continue
            data.columns = ['__'.join(col) for col in data.columns.to_flat_index()]
            data_sdmx[source+"__"+dflow] = data

    if len(data_sdmx.keys()) is None:
        return

    df = pd.concat(data_sdmx, axis=1)
    df.columns = ['_'.join(col) for col in df.columns.to_flat_index()]
    return df

In [ ]:
show_doc(load_SDMX_data)

<h4 id="load_SDMX_data" class="doc_header"><code>load_SDMX_data</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>load_SDMX_data</code>(**`sources`**, **`keys`**, **`params`**, **`verbose`**=*`True`*)

Loads datasets from SDMX.

In [ ]:
df = load_SDMX_data(sources={'ECB': 'CISS', 'BIS': 'WS_CBPOL_D'}, keys={'FREQ': 'D'}, params={'startPeriod': 2003})

assert type(df) == pd.DataFrame
assert df.shape[0] > 0
assert df.shape[1] > 0

Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...


2022-06-01 01:43:59,553 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from BIS's dataflow 'WS_CBPOL_D' - Policy rates daily...


### To be deprecated

The function `load_EURFX_data` is a helper function to download a test dataset containing real life data. This dataset was chosen due to the assumption that most users have at least an intuitive understanding of what a foreign exchange is: the price of changing one currency for the other. This example dataset does not imply this data is more or less relevant than others; it is used only for pedagogical purposes.
> Note: This function will be deprecated in `gingado` version 0.0.2.

In [ ]:
#hide
#export
import pandasdmx as sdmx
import warnings

def load_EURFX_data(startYear=2003, lags=1, jump=0, keep_contemporaneous_X=True):
    "Loads a real-life dataset for testing use cases."
    warnings.warn(message="Function 'load_EURFX_data' will no longer be present after gingado v0.0.2. Use 'load_SDMX_data(source={'ECB': 'EXR'}) instead.", category=DeprecationWarning, stacklevel=2)
    ecb = sdmx.Request('ECB')
    exr_msg = ecb.dataflow('EXR')
    exr_flow = exr_msg.dataflow.EXR
    dsd = exr_flow.structure
    key = {
    "CURRENCY": ['EUR', 'AUD', 'BRL', 'CAD', 'CHF', 'GBP', 'JPY', 'SGD', 'USD'],
    "FREQ": 'D'
    }
    params = {"startPeriod": startYear}
    data_msg = ecb.data('EXR', key=key, params=params, dsd=dsd)
    df = sdmx.to_pandas(data_msg.data[0], datetime='TIME_PERIOD')
    df = df.droplevel(['FREQ', 'CURRENCY_DENOM', 'EXR_TYPE', 'EXR_SUFFIX'], axis=1).dropna(how='all')
    
    if lags or jump:
        df = Lag(lags=lags, jump=jump, keep_contemporaneous_X=keep_contemporaneous_X).fit_transform(df)
    return df

In [ ]:
show_doc(load_EURFX_data)

<h4 id="load_EURFX_data" class="doc_header"><code>load_EURFX_data</code><a href="__main__.py#L6" class="source_link" style="float:right">[source]</a></h4>

> <code>load_EURFX_data</code>(**`startYear`**=*`2003`*, **`lags`**=*`1`*, **`jump`**=*`0`*, **`keep_contemporaneous_X`**=*`True`*)

Loads a real-life dataset for testing use cases.

In [ ]:
EUR_FX = load_EURFX_data()

assert type(EUR_FX) == pd.DataFrame
assert EUR_FX.shape[0] > 0
assert EUR_FX.shape[1] > 0

EUR_FX

/var/folders/b9/p8z57lqd55xfk68xz34dg0s40000gn/T/ipykernel_32613/429544813.py:1: DeprecationWarning: Function 'load_EURFX_data' will no longer be present after gingado v0.0.2. Use 'load_SDMX_data(source={'ECB': 'EXR'}) instead.
  EUR_FX = load_EURFX_data()


,AUD,BRL,CAD,CHF,GBP,JPY,SGD,USD,AUD_lag_1,BRL_lag_1,CAD_lag_1,CHF_lag_1,GBP_lag_1,JPY_lag_1,SGD_lag_1,USD_lag_1
TIME_PERIOD,,,,,,,,,,,,,,,,
2003-01-03,1.8440,3.6112,1.6264,1.4555,0.65000,124.56,1.8132,1.0392,1.8554,3.6770,1.6422,1.4528,0.65200,124.40,1.8188,1.0446
2003-01-06,1.8281,3.5145,1.6383,1.4563,0.64950,124.40,1.8210,1.0488,1.8440,3.6112,1.6264,1.4555,0.65000,124.56,1.8132,1.0392
2003-01-07,1.8160,3.5139,1.6257,1.4565,0.64960,124.82,1.8155,1.0425,1.8281,3.5145,1.6383,1.4563,0.64950,124.40,1.8210,1.0488
2003-01-08,1.8132,3.4405,1.6231,1.4586,0.64950,124.90,1.8102,1.0377,1.8160,3.5139,1.6257,1.4565,0.64960,124.82,1.8155,1.0425
2003-01-09,1.8172,3.4915,1.6371,1.4597,0.65300,125.16,1.8244,1.0507,1.8132,3.4405,1.6231,1.4586,0.64950,124.90,1.8102,1.0377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-25,1.5126,5.1736,1.3720,1.0269,0.85295,135.34,1.4676,1.0656,1.5152,5.1793,1.3714,1.0334,0.85750,136.49,1.4722,1.0720
2022-05-26,1.5110,5.1741,1.3715,1.0283,0.85073,135.95,1.4709,1.0697,1.5126,5.1736,1.3720,1.0269,0.85295,135.34,1.4676,1.0656
2022-05-27,1.4995,5.0959,1.3661,1.0258,0.84875,136.05,1.4679,1.0722,1.5110,5.1741,1.3715,1.0283,0.85073,135.95,1.4709,1.0697
